### Halı Şube Satış Analiz

In [1]:
import unicodedata
import datetime as dt
import os
import sys
import re
import numpy as np
import pandas as pd
import math
from pathlib import Path
import nltk
from nltk import word_tokenize 
import shutil
from os.path import isfile, join
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import locale
from locale import atof
from geopy.geocoders import Nominatim
from geopy.point import Point
from geopy.exc import GeocoderTimedOut
from functools import reduce

#### Read Data

In [39]:
# df1 = pd.read_excel(r"C:\Users\user\Downloads\A 08.08.23-08.11.23.xlsx")
# df1

In [40]:
# df2 = pd.read_excel(r"C:\Users\user\Downloads\B 08.11.23-08.02.24.xlsx")
# df2

In [41]:
# df3 = pd.read_excel(r"C:\Users\user\Downloads\C 08.02.24-08.05.24.xlsx")
# df3

In [42]:
# df4 = pd.read_excel(r"C:\Users\user\Downloads\D 08.05.24-08.08.24.xlsx")
# df4

In [43]:
# df_all = pd.concat([df1,df2,df3,df4], axis=0)
# df_all

In [44]:
# df_all.info()

In [45]:
# df_all["S.Kodu"].nunique()

In [46]:
# df_all["Adı ve Soyadı"].nunique()

In [47]:
# df_all.to_excel(r"C:\Users\user\Downloads\Halı Şube Satış.xlsx", index=False)

In [48]:
sales_files = glob.glob(r"C:\Users\user\Desktop\Data Analysis\TurkuaHome\Sales_Report\Data\Satış\08.08.2023-08.08.2024\*.xlsx")
sales_files

['C:\\Users\\user\\Desktop\\Data Analysis\\TurkuaHome\\Sales_Report\\Data\\Satış\\08.08.2023-08.08.2024\\A 08.08.23-08.11.23.xlsx',
 'C:\\Users\\user\\Desktop\\Data Analysis\\TurkuaHome\\Sales_Report\\Data\\Satış\\08.08.2023-08.08.2024\\B 08.11.23-08.02.24.xlsx',
 'C:\\Users\\user\\Desktop\\Data Analysis\\TurkuaHome\\Sales_Report\\Data\\Satış\\08.08.2023-08.08.2024\\C 08.02.24-08.05.24.xlsx',
 'C:\\Users\\user\\Desktop\\Data Analysis\\TurkuaHome\\Sales_Report\\Data\\Satış\\08.08.2023-08.08.2024\\D 08.05.24-08.08.24.xlsx']

In [49]:
df_all = pd.DataFrame()
for file in sales_files:
    df_var = pd.read_excel(f"{file}")
    df_all = pd.concat([df_all,df_var], axis=0)
df_all

,S.Kodu,Ü.Kodu,Cari Kodu,Firma,Adı ve Soyadı,S.Tar.,Ürün,Kesim,G.EN,EN,...,M2,Fiyat,İnd.,% İnd.,KDV,Toplam,Durum,Açıklama,Fatura,İşlem
0,2769,11380,NaN,Orhan Eroğlu,Adem ŞENTÜRK,2023-11-07,Neptün,Dikdörtgen,70.0,80.0,...,2.00,580.00,0.00,0.0,0.0,580.00,Kargoya Verildi,NaN,Cari,Detay
1,2769,11379,NaN,Orhan Eroğlu,Adem ŞENTÜRK,2023-11-07,Neptün,Dikdörtgen,70.0,80.0,...,0.86,248.24,0.00,0.0,0.0,248.24,Kargoya Verildi,NaN,Cari,Detay
2,2769,11378,NaN,Orhan Eroğlu,Adem ŞENTÜRK,2023-11-07,Neptün,Dikdörtgen,75.0,80.0,...,0.80,232.00,0.00,0.0,0.0,232.00,Kargoya Verildi,NaN,Cari,Detay
3,2768,11377,NaN,Orhan Eroğlu,Halil YİĞİT,2023-11-07,Orkide,Dikdörtgen,160.0,160.0,...,3.68,1232.80,123.28,10.0,0.0,1109.52,Kargoya Verildi,NaN,Cari,Detay
4,2768,11376,NaN,Orhan Eroğlu,Halil YİĞİT,2023-11-07,Orkide,Dikdörtgen,160.0,160.0,...,3.68,1232.80,123.28,10.0,0.0,1109.52,Kargoya Verildi,NaN,Cari,Detay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13082,11450,37574,120.01.785,Musa Kurt,SENA HALI,2024-05-12,Nergis,Dikdörtgen,160.0,160.0,...,3.92,1195.21,119.52,10.0,0.0,1075.69,Kargoya Verildi,Renk tonu aynı olacak. Müşteri kargo.,Cari,Detay
13083,11450,37573,120.01.785,Musa Kurt,SENA HALI,2024-05-12,Nergis,Dikdörtgen,160.0,160.0,...,4.40,1341.56,134.16,10.0,0.0,1207.40,Kargoya Verildi,Renk tonu aynı olacak. Müşteri kargo.,Cari,Detay
13084,11450,37572,120.01.785,Musa Kurt,SENA HALI,2024-05-12,Nergis,Dikdörtgen,200.0,200.0,...,4.96,1512.30,151.23,10.0,0.0,1361.07,Kargoya Verildi,Renk tonu aynı olacak. Müşteri kargo.,Cari,Detay
13085,10956,37508,120.01.970,Hakan Nefesli,ERARSLANLAR HALI VE EV TEKSTİLİ,2024-05-23,Nergis,Dikdörtgen,160.0,160.0,...,3.68,1122.03,112.20,10.0,0.0,1009.83,Kargoya Verildi,NaN,Cari,Detay


In [50]:
# df_all.to_excel(r"C:\Users\user\Downloads\Halı Şube Satış (08.08.2023-08.08.2024).xlsx", index=False)

In [4]:
# trendyol_files = glob.glob(r"C:\Users\user\Desktop\RF\*\*.xlsx")
# trendyol_files

['C:\\Users\\user\\Desktop\\RF\\2021 ty\\2021-03.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2021 ty\\2021-04.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2021 ty\\2021-05.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2021 ty\\2021-06.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2021 ty\\2021-07.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2021 ty\\2021-08.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2021 ty\\2021-09.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2021 ty\\2021-10.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2021 ty\\2021-11.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2021 ty\\2021-12.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2022 ty\\2022-01.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2022 ty\\2022-02.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2022 ty\\2022-03.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2022 ty\\2022-04.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2022 ty\\2022-05.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2022 ty\\2022-06.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2022 ty\\2022-07.xlsx',
 'C:\\Users\\user\\Desktop\\RF\\2022 ty\\2022-08

In [5]:
# df_all2 = pd.DataFrame()
# for file in trendyol_files:
#     df_var = pd.read_excel(f"{file}")
#     df_all2 = pd.concat([df_all2,df_var], axis=0)
# df_all2

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's de

,Barkod,Paket No,Kargo Firması,Sipariş Tarihi,Termin Süresinin Bittiği Tarih,Kargoya Teslim Tarihi,Kargo Kodu,Sipariş Numarası,Alıcı,Teslimat Adresi,...,ETGB Tarihi,Yaş,Cinsiyet,Kargo Partner İsmi,2.Teslimat Paketi Statüsü,2.Teslimat Takip Numarası,Teslimat Numarası,Fatura No,Fatura Tarihi,Ülke
0,4710562241983,440014510,Aras Kargo Marketplace,06.03.2021 01:16,10.03.2021 13:17,08.03.2021 17:17,7.264934e+12,526157408,Bekir Kaya,Kalyon sokak. NO:33 KARDESLER SPOT DUKKANI Av...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8699375041033,442715722,Aras Kargo Marketplace,08.03.2021 14:38,10.03.2021 14:38,08.03.2021 18:38,7.264953e+12,528900731,gul kilic,beu hastanesi biyokimya lab Kozlu Zonguldak,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,604808209209,442796436,Aras Kargo Marketplace,08.03.2021 16:16,10.03.2021 16:16,09.03.2021 18:24,7.264954e+12,528989713,Rana Nuritdinova,"Büyükdere caddesi, Akıncı Bayırı Sokak, no 12....",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8699375041057,443824504,Aras Kargo Marketplace,09.03.2021 07:59,17.03.2021 07:59,09.03.2021 17:59,7.264964e+12,530090552,selda özme,PTT Genel Müdürlüğü Ek Hizmet Binası E Blok Sa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,739339869636,444908321,Aras Kargo Marketplace,09.03.2021 18:41,17.03.2021 18:41,10.03.2021 18:41,7.264975e+12,531206048,Elvin Gültekin,"Demirciköy mahallesi, Plaj Yolu Caddesi, Akgöl...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16074,8693807247222,2663606110,Trendyol Express Marketplace,31.12.2024 23:56,02.01.2025 23:59,02.01.2025 11:52,7.330020e+15,9899040307,Betül Baytok,2963. sokak no 9 daire 5 Kepez Antalya,...,NaN,NaN,Kadın,NaN,NaN,NaN,9.111782e+09,NaN,NaN,Türkiye
16075,8693807247222,2663606759,MNG Kargo Marketplace,31.12.2024 23:57,02.01.2025 23:59,02.01.2025 14:35,7.280020e+15,9899040999,Cemile Doğan,meşrutiyet sokak apartman 7 kat 3 numara 4 Se...,...,NaN,NaN,Kadın,NaN,NaN,NaN,9.111784e+09,NaN,NaN,Türkiye
16076,20074132023456,2663606759,MNG Kargo Marketplace,31.12.2024 23:57,02.01.2025 23:59,02.01.2025 14:35,7.280020e+15,9899040999,Cemile Doğan,meşrutiyet sokak apartman 7 kat 3 numara 4 Se...,...,NaN,NaN,Kadın,NaN,NaN,NaN,9.111784e+09,NaN,NaN,Türkiye
16077,8693807257696,2663605096,Trendyol Express Marketplace,31.12.2024 23:57,02.01.2025 23:59,02.01.2025 10:51,7.330020e+15,9899041208,Emine Adiyaman,Mevlana mah yıldırım Beyazıt caddesi Carrefour...,...,NaN,NaN,Belirtilmemiş,NaN,NaN,NaN,9.111785e+09,NaN,NaN,Türkiye


In [6]:
# df_all2.columns

Index(['Barkod', 'Paket No', 'Kargo Firması', 'Sipariş Tarihi',
       'Termin Süresinin Bittiği Tarih', 'Kargoya Teslim Tarihi', 'Kargo Kodu',
       'Sipariş Numarası', 'Alıcı', 'Teslimat Adresi', 'İl', 'İlçe',
       'Ürün Adı', 'Fatura Adresi', 'Alıcı - Fatura Adresi', 'Sipariş Statüsü',
       'E-Posta', 'Komisyon Oranı', 'Marka', 'Stok Kodu', 'Adet',
       'Birim Fiyatı', 'Satış Tutarı', 'İndirim Tutarı',
       'Trendyol İndirim Tutarı', 'Faturalanacak Tutar', 'Butik Numarası',
       'Teslim Tarihi', 'Kargodan alınan desi', 'Hesapladığım desi',
       'Faturalanan Kargo Tutarı', 'Alternatif Teslimat Statüsü',
       'Kurumsal Faturalı Sipariş', 'Vergi Kimlik Numarası', 'Vergi Dairesi',
       'Şirket İsmi', 'Arkadaşlarınla Al Siparişi', 'Fatura',
       'Müşteri Sipariş Adedi', 'Mikro İhracat', 'ETGB No', 'ETGB Tarihi',
       'Yaş', 'Cinsiyet', 'Kargo Partner İsmi', '2.Teslimat Paketi Statüsü',
       '2.Teslimat Takip Numarası', 'Teslimat Numarası', 'Fatura No',
       'Fatu

In [10]:
# df_all2.to_csv(r"C:\Users\user\Desktop\Trendyol_Excel_Birleştirme.csv", index=False)

In [9]:
# df_all2.to_excel(r"C:\Users\user\Desktop\Trendyol_Excel_Birleştirme.xlsx", index=False)

In [51]:
df_select = df_all[["Firma","Adı ve Soyadı","S.Tar.","Ürün","Kesim","Saçak","Desen","Adet","M2","Fiyat","İnd.","KDV","Toplam","Durum"]]
df_select

,Firma,Adı ve Soyadı,S.Tar.,Ürün,Kesim,Saçak,Desen,Adet,M2,Fiyat,İnd.,KDV,Toplam,Durum
0,Orhan Eroğlu,Adem ŞENTÜRK,2023-11-07,Neptün,Dikdörtgen,Uygun Saçak,PLS-Gri,1.0,2.00,580.00,0.00,0.0,580.00,Kargoya Verildi
1,Orhan Eroğlu,Adem ŞENTÜRK,2023-11-07,Neptün,Dikdörtgen,Uygun Saçak,PLS-Gri,1.0,0.86,248.24,0.00,0.0,248.24,Kargoya Verildi
2,Orhan Eroğlu,Adem ŞENTÜRK,2023-11-07,Neptün,Dikdörtgen,Uygun Saçak,PLS-Gri,1.0,0.80,232.00,0.00,0.0,232.00,Kargoya Verildi
3,Orhan Eroğlu,Halil YİĞİT,2023-11-07,Orkide,Dikdörtgen,Sabit Saçak,ORK0101-Gri,1.0,3.68,1232.80,123.28,0.0,1109.52,Kargoya Verildi
4,Orhan Eroğlu,Halil YİĞİT,2023-11-07,Orkide,Dikdörtgen,Sabit Saçak,ORK0104-Kahve,1.0,3.68,1232.80,123.28,0.0,1109.52,Kargoya Verildi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13082,Musa Kurt,SENA HALI,2024-05-12,Nergis,Dikdörtgen,Sabit Saçak,NGS0101-Gri,1.0,3.92,1195.21,119.52,0.0,1075.69,Kargoya Verildi
13083,Musa Kurt,SENA HALI,2024-05-12,Nergis,Dikdörtgen,Sabit Saçak,NGS0101-Gri,1.0,4.40,1341.56,134.16,0.0,1207.40,Kargoya Verildi
13084,Musa Kurt,SENA HALI,2024-05-12,Nergis,Dikdörtgen,Sabit Saçak,NGS0101-Gri,1.0,4.96,1512.30,151.23,0.0,1361.07,Kargoya Verildi
13085,Hakan Nefesli,ERARSLANLAR HALI VE EV TEKSTİLİ,2024-05-23,Nergis,Dikdörtgen,Sabit Saçak,NGS0104-Kahve,1.0,3.68,1122.03,112.20,0.0,1009.83,Kargoya Verildi


In [52]:
df_select.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47485 entries, 0 to 13086
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Firma          47485 non-null  object        
 1   Adı ve Soyadı  47485 non-null  object        
 2   S.Tar.         47485 non-null  datetime64[ns]
 3   Ürün           47481 non-null  object        
 4   Kesim          47485 non-null  object        
 5   Saçak          47485 non-null  object        
 6   Desen          47296 non-null  object        
 7   Adet           47484 non-null  float64       
 8   M2             47485 non-null  float64       
 9   Fiyat          47485 non-null  float64       
 10  İnd.           47485 non-null  float64       
 11  KDV            47485 non-null  float64       
 12  Toplam         47485 non-null  float64       
 13  Durum          47485 non-null  object        
dtypes: datetime64[ns](1), float64(6), object(7)
memory usage: 5.4+ MB


In [53]:
df_select = df_select.dropna(subset=["Firma","Adı ve Soyadı","S.Tar.","Ürün","Kesim","Saçak","Desen","Adet","M2","Fiyat","İnd.","KDV","Toplam","Durum"], axis=0)
df_select

,Firma,Adı ve Soyadı,S.Tar.,Ürün,Kesim,Saçak,Desen,Adet,M2,Fiyat,İnd.,KDV,Toplam,Durum
0,Orhan Eroğlu,Adem ŞENTÜRK,2023-11-07,Neptün,Dikdörtgen,Uygun Saçak,PLS-Gri,1.0,2.00,580.00,0.00,0.0,580.00,Kargoya Verildi
1,Orhan Eroğlu,Adem ŞENTÜRK,2023-11-07,Neptün,Dikdörtgen,Uygun Saçak,PLS-Gri,1.0,0.86,248.24,0.00,0.0,248.24,Kargoya Verildi
2,Orhan Eroğlu,Adem ŞENTÜRK,2023-11-07,Neptün,Dikdörtgen,Uygun Saçak,PLS-Gri,1.0,0.80,232.00,0.00,0.0,232.00,Kargoya Verildi
3,Orhan Eroğlu,Halil YİĞİT,2023-11-07,Orkide,Dikdörtgen,Sabit Saçak,ORK0101-Gri,1.0,3.68,1232.80,123.28,0.0,1109.52,Kargoya Verildi
4,Orhan Eroğlu,Halil YİĞİT,2023-11-07,Orkide,Dikdörtgen,Sabit Saçak,ORK0104-Kahve,1.0,3.68,1232.80,123.28,0.0,1109.52,Kargoya Verildi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13082,Musa Kurt,SENA HALI,2024-05-12,Nergis,Dikdörtgen,Sabit Saçak,NGS0101-Gri,1.0,3.92,1195.21,119.52,0.0,1075.69,Kargoya Verildi
13083,Musa Kurt,SENA HALI,2024-05-12,Nergis,Dikdörtgen,Sabit Saçak,NGS0101-Gri,1.0,4.40,1341.56,134.16,0.0,1207.40,Kargoya Verildi
13084,Musa Kurt,SENA HALI,2024-05-12,Nergis,Dikdörtgen,Sabit Saçak,NGS0101-Gri,1.0,4.96,1512.30,151.23,0.0,1361.07,Kargoya Verildi
13085,Hakan Nefesli,ERARSLANLAR HALI VE EV TEKSTİLİ,2024-05-23,Nergis,Dikdörtgen,Sabit Saçak,NGS0104-Kahve,1.0,3.68,1122.03,112.20,0.0,1009.83,Kargoya Verildi


In [54]:
# df_select.to_excel(r"C:\Users\user\Downloads\Halı Şube Satış Select (08.08.2023-08.08.2024).xlsx", index=False)

In [55]:
df_select_group = df_select.groupby(["Firma","Ürün"])["Adet"].sum().reset_index()
df_select_group_sort = df_select_group.sort_values(["Firma","Adet"], ascending=[True,False])
df_select_group_sort = df_select_group_sort.reset_index(drop=True)
df_select_group_sort

,Firma,Ürün,Adet
0,Abdulbaki Ayık,Agat,2778.0
1,Abdulbaki Ayık,Lilyum,654.0
2,Abdulbaki Ayık,Nergis,413.0
3,Abdulbaki Ayık,Mimoza,322.0
4,Abdulbaki Ayık,Mira,286.0
...,...,...,...
760,Şaban Silahşör,Luna,3.0
761,Şaban Silahşör,Mimoza,3.0
762,Şaban Silahşör,Orkide,3.0
763,Şaban Silahşör,Berlin,2.0


In [56]:
df_select_group_sort.to_excel(r"C:\Users\user\Downloads\Halı Şube Satış Select Adet (08.08.2023-08.08.2024).xlsx", index=False)

In [57]:
df_select_group2 = df_select.groupby("Ürün")["Adet"].sum().reset_index()
df_select_group_sort2 = df_select_group2.sort_values("Adet", ascending=False)
df_select_group_sort2 = df_select_group_sort2.reset_index(drop=True)
df_select_group_sort2

,Ürün,Adet
0,Mimoza,33538.0
1,Agat,10581.0
2,Mira,9111.0
3,Havana,7970.0
4,Neptün,7937.0
5,Luna,6165.0
6,Angora,5699.0
7,Gri Dot Taban 160x230 Kampanya,5415.0
8,Orkide,5299.0
9,Lilyum,2898.0


In [58]:
df_select_group_sort2.to_excel(r"C:\Users\user\Downloads\Halı Satış Select Adet (08.08.2023-08.08.2024).xlsx", index=False)

In [59]:
df_select_group3 = df_select.groupby("Ürün")["Fiyat"].sum().reset_index()
df_select_group_sort3 = df_select_group3.sort_values("Fiyat", ascending=False)
df_select_group_sort3 = df_select_group_sort3.reset_index(drop=True)
df_select_group_sort3

,Ürün,Fiyat
0,Havana,9494182.37
1,Mira,8309587.42
2,Orkide,6537418.68
3,Luna,5413475.00
4,Angora,4763342.16
5,Agat,4079131.09
6,Lotus,3158600.41
7,Mimoza,3002120.21
8,Lilyum,2640271.87
9,Neptün,2518045.61


In [60]:
df_select_group_sort3.to_excel(r"C:\Users\user\Downloads\Halı Satış Select TL (08.08.2023-08.08.2024).xlsx", index=False)

In [61]:
df_select_group4 = df_select.groupby("Ürün")["M2"].sum().reset_index()
df_select_group_sort4 = df_select_group4.sort_values("M2", ascending=False)
df_select_group_sort4 = df_select_group_sort4.reset_index(drop=True)
df_select_group_sort4

,Ürün,M2
0,Mira,25194.85
1,Mimoza,21797.70
2,Agat,18273.20
3,Luna,16183.76
4,Orkide,15730.58
5,Havana,14088.50
6,Angora,8860.05
7,Lilyum,8397.59
8,Neptün,7398.92
9,Lotus,7318.86


In [62]:
df_select_group_sort4.to_excel(r"C:\Users\user\Downloads\Halı Satış Select M2 (08.08.2023-08.08.2024).xlsx", index=False)